# 초모수와 모델 검증

지도학습 모델을 적용하는 기본 단계는 아래와 같다.

1. 모델 클래스 선택
2. 모델 초모수 선택
3. 모델을 훈련 데이터에 적합
4. 모델을 사용해 새 데이터에 대한 레이블 예측

처음 두 단계인 모델 선택과 초모수 선택이 아마 이 도구와 기법을 효과적으로 사용하는 데 가장 중요한 부분일 것이다. 정보에 입각한 선택을 하려면 선택한 모델과 초모수가 데이터에 잘 적합되는지 검증할 방법이 필요하다. 간단한 것처럼 들리지만 효과적인 검증을 위해서 피해야 할 몇가지 주의 사항이 있다.

## 1. 모델 검증에 대한 고려 사항

이론적으로 모델 검증은 매우 단순하다. 모델과 모델의 초모수를 선택한 후, 훈련 데이터 일부에 이를 적용하고 예측값을 알려진 값과 비교해서 이 모델이 얼마나 효과적인지 추정할 수 있다.

먼저 순진한 방식으로 모델을 검증하면 왜 실패하는지 보여주고좀 더 견고한 모델 평가를 위해 검정 표본과 교차 검증을 사용해보자.

### 잘못된 방식의 모델 검증

~ 생략 ~

### 올바른 방식의 모델 검증 : 검증 표본

모델의 성능을 제대로 알려면 검증 표본(holdout set)을 사용하면 된다.

즉, 모델의 훈련 데이터에서 데이터의 일부를 빼내 그것을 모델 성능을 확인하는 검정 표본으로 사용하는 것이다. 이 분리 작업에는 scikit-learn의 `train_test_split` 유틸리티를 사용하면 된다.

### 교차 검증을 통한 모델 검증

모델 검증에 검정 표본을 사용할 때의 한 가지 단점은 모델을 훈련시킬 데이터의 일부를 잃게된다는 것이다. 앞의 경우에 데이터 세트의 반은 모델을 훈련시키는데 기여하지 못한다. 이는 특히 초기 훈련 데이터가 작은 경우에는 적절하지 않은 방법으로, 문제가 발생할 수 있다.

이 문제를 해결하기 위한 한가지 방법은 **교차 검증**(cross-validation)을 사용하는 것이다. 즉, 데이터의 각 하위 집합이 훈련 자료와 검증 자료로 사용되도록 일련의 적합을 수행하는 것이다.

여기서도 마찬가지로 사이킷런의 `cross_val_score`를 사용하면 간결하게 수행할 수 있다.


In [ ]:
from sklearn.cross_validation import cross_val_score

cross_val_score(model, X, y, cv=5)

데이터의 하위 집합별로 검증을 반복함으로써 알고리즘 성능에 대해 더 나은 판단을 할 수 있다.

사이킷런은 특정 상황에 유용한 수많은 교차 검증 방식을 구현하고 있다. 이 방식은 cross_validation 모듀ㅜㄹ의 반복자를 통해 구현된다. 예를 들어, 데이터 점의 개수와 같은 수만큼 반복하는 교차 검증을 하는 극단적인 경우를 원할수도 있다. 다시 말하면 검증을 시행할 때마다 한 점을 제외한 모든 점에 대해 훈련하는 것이다. 이러한 유형의 교차 검증을 **단일 관측치 제거 방식**(leave-one-out) 교차검증이라고 한다.

In [ ]:
from sklearn.cross_validation import LeaveOneOut

scores = cross_val_score(model, X, y, cv=LeaveOneOut(len(X)))

## 최적의 모델 선택하기

검증과 교차 검증의 기본 내용을 살펴봤으니 이제 모델과 초모수 선택과 관련해 더 깊이있게 알아보자. 이 이슈는 실제 머신러닝에서 가장 중요한 측면에 속하지만, 머신러닝 입문서에서는 그 정보를 건너 뛰는 경우가 자주 있다.

다음과 같은 질문을 하는 것이 가장 중요하다. *모델의 성능이 저조하다면 어떻게 개선할 것인가?* 이 문제를 해결하는 몇가지 답이 있다.

- 더 복잡하거나 더 유연한 모델 사용
- 덜 복잡하거나 덜 유연한 모델 사용
- 더 많은 훈련 표본 수집
- 각 표본에 특징을 추가하기 위해 더 많은 데이터 수집

이 질문에 대한 답은 종종 직관에 어긋난다. 특히 더 복잡한 모델을 사용해서 결과가 악화되는 경우도 있고 더 많은 훈련 표본을 추가해도 결과가 개선되지 않을 수 있다. 어느 단계가 모델을 개선할 수 있을지 결정하는 능력이 성공한 머신러닝 실무자와 실패한 실무자를 구분짓는다.


## 편향 - 분산 트레이드오프

근본적으로 '최고의 모델'을 선택하는 것은 편향과 분산 사이의 트레이드 오프에서 가장 효율적인 점을 찾는 것이다.

- 고 편향 모델은 데이터를 과소적합한다.
- 고분산 모델은 데이터를 과적합한다.

